# Quick Recap

CrewAI is a Python framework that facilitates the orchestration of autonomous AI agents, each equipped with specialized tools to collaboratively tackle complex tasks. It enables efficient multi-agent workflows, where agents communicate, share information, and execute tasks with strategic coordination.

### **Core Concepts of CrewAI**
- **Crew:** A collaborative team of AI agents working together towards a shared objective.
- **Agent:** An autonomous unit with defined roles, goals, and tools that enable decision-making, memory retention, and delegation.

### **Key Attributes of an Agent**
- **Role:** Defines the agent’s function (e.g., 'Analyst', 'Customer Service Rep').
- **Goal:** Establishes the agent’s primary objective.
- **Backstory:** Adds context and depth to an agent’s persona, improving interactions.

### **Tools in CrewAI**
CrewAI provides built-in tools for various functionalities, including:
- **Web Scraping (ScrapeWebsiteTool):** Extracts data from websites.
- **File Management (FileReadTool, FileWriteTool):** Reads and writes files.
- **Database Interaction (PGSearchTool):** Connects agents to PostgreSQL databases, and searches related content from the database.
- **Image Generation (DallETool):** Creates images using OpenAI’s DALL·E.
- **Search Integration (SerperDevTool):** Conducts research with live web data.

Agents can also be equipped with **custom tools** by subclassing `BaseTool` to extend their capabilities.

### **Collaboration in CrewAI**
Multi-agent collaboration enables complex problem-solving, with agents communicating, coordinating, negotiating, and planning together.

### **Memory in CrewAI**
CrewAI integrates a structured memory system that allows agents to retain and recall information:
- **Short-Term Memory:** Stores recent interactions for context-aware responses.
- **Long-Term Memory:** Retains knowledge for continuous learning.
- **Entity Memory:** Captures structured data about people, places, or concepts.
- **Contextual Memory:** Combines different memory types for coherent, long-term interaction management.


# Knowledge

In CrewAI, the **Knowledge** system enables AI agents to access and utilize external information sources during their tasks. This system functions as a reference library, enhancing agents' capabilities with domain-specific information, supporting decisions with real-world data, maintaining context across conversations, and grounding responses in factual information.

**Supported Knowledge Sources:**

- **Text Sources:**
  - Raw strings
  - Text files (.txt)
  - PDF documents

- **Structured Data:**
  - CSV files
  - Excel spreadsheets
  - JSON documents

**Important Knowledge Parameters:**

- **`sources`** (List[BaseKnowledgeSource], required): A list of knowledge sources providing content to be stored and queried, including PDF, CSV, Excel, JSON, text files, or string content.

- **`collection_name`** (str, optional): The name of the collection where the knowledge will be stored, used to identify different sets of knowledge. Defaults to "knowledge" if not provided.

- **`storage`** (Optional[KnowledgeStorage], optional): Custom storage configuration for managing how the knowledge is stored and retrieved. If not provided, a default storage will be created.

### Example 1: **Using String-Based Knowledge**

You can provide agents with specific information using raw strings.


```python
from crewai import Agent, Task, Crew
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource

# Create a knowledge source
content = "Our product, the XPS 13, has a storage capacity of 512GB SSD."
string_source = StringKnowledgeSource(content=content)

# Create an agent with the knowledge source
support_agent = Agent(
    role="Technical Support Specialist",
    goal="Provide accurate product information and support.",
    backstory="You are a good support specialist",
    knowledge_sources=[string_source]
)

# Define a task for the agent
support_task = Task(
    description="Answer the customer's question about product specifications.",
    agent=support_agent
)

# Create and run the crew
crew = Crew(
    agents=[support_agent],
    tasks=[support_task]
)

# Get the agent's response
result = crew.kickoff(inputs={"question": "What is the storage capacity of the XPS 13?"})
print(result)
```


In this example, the agent utilizes the provided string-based knowledge to answer the customer's question.

### Example 2: **Using File-Based Knowledge**

Agents can also extract information from various file types, such as PDFs.


```python
from crewai import Agent, Task, Crew
from crewai.knowledge.source.pdf_knowledge_source import PDFKnowledgeSource

# Load a PDF knowledge source
pdf_source = PDFKnowledgeSource(
    file_path="product_manual.pdf",  # Ensure this file is in the 'knowledge' directory
    chunk_size=4000,
    chunk_overlap=200
)

# Create an agent with the PDF knowledge source
support_agent = Agent(
    role="Technical Support Specialist",
    goal="Provide accurate product information and support.",
    backstory="You are a good support specialist",
    knowledge_sources=[pdf_source]
)

# Define a task for the agent
support_task = Task(
    description="Answer the customer's question about product usage.",
    agent=support_agent
)

# Create and run the crew
crew = Crew(
    agents=[support_agent],
    tasks=[support_task]
)

# Get the agent's response
result = crew.kickoff(inputs={"question": "How do I reset the XPS 13 to factory settings?"})
print(result)
```


Here, the agent references the content of the PDF manual to provide accurate support information.

### Example 3: **Creating Custom Knowledge Sources**

You can extend CrewAI's capabilities by creating custom knowledge sources. For instance, fetching and processing space news articles:


```python
from crewai.knowledge.source.base_knowledge_source import BaseKnowledgeSource
import requests
from typing import Dict, Any
from pydantic import Field

class SpaceNewsKnowledgeSource(BaseKnowledgeSource):
    """Knowledge source that fetches data from a Space News API."""

    api_endpoint: str = Field(description="API endpoint URL")
    limit: int = Field(default=10, description="Number of articles to fetch")

    def load_content(self) -> Dict[Any, str]:
        """Fetch and format space news articles."""
        try:
            response = requests.get(f"{self.api_endpoint}?limit={self.limit}")
            response.raise_for_status()
            data = response.json()
            articles = data.get('results', [])
            formatted_data = self._format_articles(articles)
            return {self.api_endpoint: formatted_data}
        except Exception as e:
            raise ValueError(f"Failed to fetch space news: {str(e)}")

    def _format_articles(self, articles: list) -> str:
        """Format articles into readable text."""
        formatted = "Space News Articles:\n\n"
        for article in articles:
            formatted += f"""
            Title: {article['title']}
            Published: {article['published_at']}
            Summary: {article['summary']}
            News Site: {article['news_site']}
            URL: {article['url']}
            """
        return formatted

# Instantiate the custom knowledge source
space_news_source = SpaceNewsKnowledgeSource(api_endpoint="https://api.spaceflightnewsapi.net/v3/articles")

# Create an agent with the custom knowledge source
news_agent = Agent(
    role="Space News Analyst",
    goal="Provide the latest updates on space exploration.",
    knowledge_sources=[space_news_source]
)

# Define a task for the agent
news_task = Task(
    description="Summarize the latest space news articles.",
    agent=news_agent
)

# Create and run the crew
crew = Crew(
    agents=[news_agent],
    tasks=[news_task]
)

# Get the agent's summary
result = crew.kickoff()
print(result)
```


In this case, the agent fetches and summarizes the latest space news articles using the custom knowledge source.

# Processes
*Think Gantt Charts*

In CrewAI, **Processes** define the workflow strategy for how tasks are managed and executed by agents, ensuring efficient collaboration and task completion. They dictate the structure and flow of operations within a crew, similar to project management methodologies in human teams.

**Types of Processes in CrewAI:**

1. **Sequential Process:**
   - Tasks are executed one after the other in a predetermined order.
   - Ideal for projects where tasks need to be completed in a specific sequence.
   - Simplifies tracking and monitoring of progress.

   *Example:*

   ```python
   from crewai import Crew, Process, Agent, Task

   # Define agents
   researcher = Agent(
       role='Researcher',
       goal='Conduct foundational research',
       backstory='An experienced researcher with a passion for uncovering insights'
   )
   analyst = Agent(
       role='Data Analyst',
       goal='Analyze research findings',
       backstory='A meticulous analyst with a knack for uncovering patterns'
   )
   writer = Agent(
       role='Writer',
       goal='Draft the final report',
       backstory='A skilled writer with a talent for crafting compelling narratives'
   )

   # Define tasks
   research_task = Task(
       description='Gather relevant data...',
       agent=researcher,
       expected_output='Raw Data'
   )
   analysis_task = Task(
       description='Analyze the data...',
       agent=analyst,
       expected_output='Data Insights'
   )
   writing_task = Task(
       description='Compose the report...',
       agent=writer,
       expected_output='Final Report'
   )

   # Form the crew with a sequential process
   report_crew = Crew(
       agents=[researcher, analyst, writer],
       tasks=[research_task, analysis_task, writing_task],
       process=Process.sequential
   )

   # Execute the crew
   result = report_crew.kickoff()
   ```


   In this example, each agent completes their task in sequence, with the output of one task serving as input for the next.

2. **Hierarchical Process:**
   - Emulates a corporate hierarchy with a manager agent overseeing task delegation and execution.
   - The manager plans, assigns tasks to agents based on their expertise, and reviews outputs.
   - Suitable for complex projects requiring dynamic task management and oversight.

   *Example:*

   ```python
   from crewai import Crew, Process, Agent

   # Define agents
   researcher = Agent(
       role='Researcher',
       goal='Conduct in-depth analysis',
       backstory='Experienced data analyst with a knack for uncovering hidden trends.'
   )
   writer = Agent(
       role='Writer',
       goal='Create engaging content',
       backstory='Creative writer passionate about storytelling in technical domains.'
   )

   # Define a custom manager agent
   manager = Agent(
       role="Project Manager",
       goal="Efficiently manage the crew and ensure high-quality task completion",
       backstory="An experienced project manager skilled in overseeing complex projects and guiding teams to success.",
       allow_delegation=True,
   )

   # Establish the crew with a hierarchical process
   project_crew = Crew(
       agents=[researcher, writer],
       manager_agent=manager,  # Use the custom manager agent
       process=Process.hierarchical,
       planning=True,
   )

   # Execute the crew
   result = project_crew.kickoff()
   ```


   In this setup, the manager agent dynamically assigns tasks to the researcher and writer, monitors progress, and ensures quality control throughout the project lifecycle.

3. **Consensual Process (Planned):**
   - Envisions collaborative decision-making among agents regarding task execution.
   - Introduces a democratic approach to task management within CrewAI.
   - Note: This process type is planned for future development and is not currently implemented in the codebase.

**Assigning a Process to a Crew:**

When creating a crew, specify the desired process type to define the execution strategy. For a hierarchical process, ensure to define `manager_llm` or `manager_agent` for the manager agent.


```python

# Example: Creating a crew with a hierarchical process
# Ensure to provide a manager_llm or manager_agent
crew = Crew(
    agents=my_agents,
    tasks=my_tasks,
    process=Process.hierarchical,
    manager_llm="gpt-4o"
    # or
    # manager_agent=my_manager_agent
)
```


*Note: Ensure `my_agents` and `my_tasks` are defined prior to creating a `Crew` object, and for the hierarchical process, either `manager_llm` or `manager_agent` is also required.*


# Planning
*Think "Review before Execution"*

In CrewAI, **Planning** enhances the efficiency and effectiveness of AI agent collaborations by enabling a structured approach to task execution. When planning is enabled, an `AgentPlanner` analyzes all crew information before each iteration, devising a step-by-step plan that is incorporated into each task description.

**Implementing Planning in CrewAI:**

To activate the planning feature, set the `planning` parameter to `True` when creating a `Crew` instance:


```python
from crewai import Crew, Agent, Task, Process

# Define agents and tasks
agents = [Agent(role='Researcher', goal='Conduct research'),
          Agent(role='Analyst', goal='Analyze data')]
tasks = [Task(description='Research market trends'),
         Task(description='Analyze quarterly data')]

# Create a crew with planning enabled
my_crew = Crew(
    agents=agents,
    tasks=tasks,
    process=Process.sequential,
    planning=True
)

# Execute the crew
my_crew.kickoff()
```


In this setup, the `AgentPlanner` formulates a detailed plan for each task before execution, guiding agents through a structured workflow.

**Specifying the Planning Language Model (LLM):**

CrewAI allows customization of the language model used for planning by setting the `planning_llm` parameter:


```python
from crewai import Crew, Agent, Task, Process

# Define agents and tasks
agents = [Agent(role='Researcher', goal='Conduct research'),
          Agent(role='Analyst', goal='Analyze data')]
tasks = [Task(description='Research market trends'),
         Task(description='Analyze quarterly data')]

# Create a crew with planning enabled and a custom LLM
my_crew = Crew(
    agents=agents,
    tasks=tasks,
    process=Process.sequential,
    planning=True,
    planning_llm="gpt-4o"
)

# Execute the crew
my_crew.kickoff()
```


Here, the `AgentPlanner` utilizes the specified `gpt-4o` model to generate the planning steps, tailoring the workflow to the chosen LLM's capabilities.

